We define a generic class InputGeneric which is inherited to InputVariable and InputVariablesGroup.
We detect a variable (InputVariable) by the presence of the key 'default' otherwise it is a group of variables (inputVariablesGroup).

In [29]:
from difflib import SequenceMatcher
import yaml

def closest_keys(inputs,searchfor):
    """Find the closest input variables from searchfor"""
    doc = []
    for (var,ratio) in inputs.match(searchfor):
        if ratio >= 0.5:
            doc.append(var)
    return doc
    #print yaml.dump(doc, default_flow_style=False, explicit_start=True)
    
class FindKeys(list):
    """List of InputVariables matching a sequence."""
    #def __new__(cls,:
    #    return list.__new__(cls) 
    def __init__(self,inputs,searchfor):
        self += closest_keys(inputs,searchfor)
        self.searchfor=searchfor
    def __str__(self):
        string='Keys which match the most with "%s"\n' % self.searchfor
        string+=str([ (c.name,c.COMMENT) for c in self])
        return string
    
class InputGeneric():
    """Generic class to define input variables or group of input variables."""
    def __init__(self,name,definition,parent=None):
        """From the dictionary import the tree of variables."""
        self.name = name
        self.parent = parent
        self.__rawdict__ = definition.copy()
        self.DESCRIPTION = self.__rawdict__.pop('DESCRIPTION','')
        self.__vars__ = self.__rawdict__.keys()
        self.__vars__.sort()
        #By default check the last level of input variables group.
        self.__safeload = False
    def __str__(self):
        """Return the full name of the variable with its parents"""
        if self.parent:
            return str(self.parent)+"."+self.name
        else:
            return self.name
    def full(self):
        """Return a full input yaml document"""
        return yaml.dump(self.todict(),default_flow_style=False)
    def load(self,string,safe=True):
        """Load a yaml dict and set the corresponding variables.
           If the option 'safe' is false, do not check if an input variable does exist."""
        if isinstance(string,str) or isinstance(string,file):
            doc = yaml.load(string)
        elif isinstance(string,dict):
            doc = string
        self.load_doc(doc,safe)
    def load_doc(self,doc,safe=True):
        """Load a yaml dict and set the corresponding variables.
           If the option 'safe' is false, do not check if an input variable does exist."""
        for (key,val) in doc.items():
            if key in self.__vars__:
                var = getattr(self,key)
                var.load_doc(val)
            elif safe and self.__safeload:
                raise KeyError("The key '%s' for '%s' does not exist!" % (key,self.name))
    def match(self,searchfor):
        """Compare a sequence to all keys"""
        matching_keys = []
        for k in self.__vars__:
            var = getattr(self,k)
            m = var.match(searchfor)
            if (isinstance(m,list)):
                matching_keys.extend(m)
            else:
                matching_keys.append(m)
        matching_keys.sort(key=lambda x: x[1])
        matching_keys.reverse()
        return matching_keys
    def minimal(self):
        """Return a minimal input yaml document"""
        return yaml.dump(self.tomin(),default_flow_style=False)
    def search(self,sequence):
        """Return a list (class FindKeys) of Inputvariables matching the sequence."""
        return FindKeys(self,sequence)
    def todict(self):
        """Translate into a dictionary"""
        return {k: getattr(self,k).todict() for k in self.__vars__}
    def tomin(self):
        """Return a minimal input yaml document"""
        doc = {k: getattr(self,k).tomin() for k in self.__vars__}
        for (key,val) in doc.items():
            if val == 'default' or val == {}:
                del doc[key]
        return doc
    def wiki(self):
        """Wiki representation of the input variables"""
        cr="\r\n"
        string="="+self.name+"="+cr
        for var in self.__vars__:
            variable=getattr(self,var)
            string+=wiki(variable)
        return string
    
class InputVariablesGroup(InputGeneric):
    """Define group of input variables."""
    def __init__(self,name,definition,parent=None):
        InputGeneric.__init__(self,name,definition,parent)
        for var in self.__rawdict__:
            if definition[var].has_key('default'):
                setattr(self,var,InputVariable(var,definition[var],self))
                #Last level of groups of input variables
                self.__safeload = True
            else:
                setattr(self,var,InputVariablesGroup(var,definition[var],self))
    
class InputVariable(InputGeneric):
    """Class of input variable"""
    def __init__(self,name,definition,parent=None):
        """Define the variable associated to the input variable in futile proposed format"""
        #Raw definiton
        InputGeneric.__init__(self,name,definition,parent)
        #Set the attribute and the meta values
        self.__meta = []
        for key in definition:
            setattr(self,key,definition[key])
            if key not in ['COMMENT', 'DESCRIPTION', 'default', 'CONDITION', 'EXCLUSIVE', 'RANGE']:
                self.__meta.append(key)
        #Set the default value (use 'default' to avoid the condition)
        self.set('default')
    def __str__(self):
        """return the full name and the value of the variable"""
        return InputGeneric.__str__(self)+": "+str(self.value)
    def load_doc(self,doc):
        """Set the value from a document"""
        self.set(doc)
    def match(self,searchfor):
        """Give the match of the input variable with a given sequence"""
        #s = SequenceMatcher(lambda x: x == " ", searchfor, 
        #                    "%s %s %s" % (self.name,self.COMMENT,self.DESCRIPTION))
        slow=searchfor.lower()
        if slow in self.COMMENT.lower():
            ratio = 1.0
        elif slow in self.DESCRIPTION.lower():
            ratio = 1.0
        else:
            ratio = SequenceMatcher(lambda x: x == " ", slow,self.name.lower()).ratio()
        return (self, ratio)
    def set(self,value,reset=True):
        """Set the value for the corresponding variables and check it."""
        #Special case for default
        if value == 'default': 
            self.value = self.default
            #Check nothing (not useful and avoid CONDITION the first time)
            return
        else:
            self.value = value
        #Check if the value is equal to a meta value and replace by the meta key
        for m in self.__meta:
            if self.value == getattr(self,m):
                self.value = m
        if self.value in self.__meta:
            pass
        #here we can check that the value is legal following the definitions
        elif hasattr(self,'EXCLUSIVE') and self.value not in self.EXCLUSIVE:
            raise TypeError('Permitted value: '+str(self.EXCLUSIVE)+ \
                            "\nValue ('"+str(self.value)+"') not admitted for variable "+str(self.name))
        elif hasattr(self,'RANGE') and (self.value < self.RANGE[0] or self.value > self.RANGE[1]):
            raise TypeError('Permitted range: '+str(self.RANGE)+ \
                            "\nValue ("+str(self.value)+") not admitted for variable "+str(self.name))
        #We check the CONDITION 
        if hasattr(self,'CONDITION'):
            cond = getattr(self,'CONDITION')
            master = getattr(self.parent,cond['MASTER_KEY'])
            #Put to default if the condition is not satisfied
            if 'WHEN' in cond and (not master.value in cond['WHEN']):
                self.value = self.default
            elif 'WHEN_NOT' in cond and (master.value in cond['WHEN_NOT']):
                self.value = self.default
        if reset: 
            self.re_set()
    
    def re_set(self):
        """Re-set all variables of the parent"""
        parent=self.parent
        for key in parent.__vars__:
            if key==self.name: continue
            var=getattr(parent,key)
            var.set(var.value,reset=False)
    
    def todict(self):
        """Return the value"""
        return self.value
    def tomin(self):
        """Return the value if it is not the default"""
        if self.value == self.default:
            return 'default'
        else:
            return self.value
    def wiki(self):
        cr="\r\n"
        string="=="+self.name+"=="+cr
        if hasattr(self,'DESCRIPTION'):
            string+=self.DESCRIPTION+cr
        elif hasattr(self,'COMMENT'):
            string+=self.COMMENT+cr
        string+="Default value: "+str(self.default)+cr
        if hasattr(self,'EXCLUSIVE'):
            for ex in self.EXCLUSIVE:
                string+='* '+str(ex)+cr
        return string


In [23]:
import yaml
isf=yaml.load('''
  isf_order: 
    COMMENT: Order of the Interpolating Scaling Function family
    DESCRIPTION: Fixes the order of the ISF family that is used for the discretization of the kernel
    default: 16
    EXCLUSIVE: [ 2, 4, 6, 8, 14, 16, 20, 24, 30, 40, 50, 60, 100]
''')
print isf

isfvar=InputVariable('isf_order',isf['isf_order'])
print isfvar.wiki()
print isfvar.todict()
a = isfvar.match('isf')
print a[0].name,a[1]

{'isf_order': {'COMMENT': 'Order of the Interpolating Scaling Function family', 'default': 16, 'DESCRIPTION': 'Fixes the order of the ISF family that is used for the discretization of the kernel', 'EXCLUSIVE': [2, 4, 6, 8, 14, 16, 20, 24, 30, 40, 50, 60, 100]}}
==isf_order==
Fixes the order of the ISF family that is used for the discretization of the kernel
Default value: 16
* 2
* 4
* 6
* 8
* 14
* 16
* 20
* 24
* 30
* 40
* 50
* 60
* 100

16
isf_order 1.0


In [24]:
docs=[a for a in yaml.load_all(open('../../psolver/src/PS_input_variables_definition.yaml','r'))]
ps=docs[0]
psolver = InputVariablesGroup('psolver',ps)
psolver.environment.epsilon.set(78.36)
print psolver.environment.epsilon.value == 78.36
print psolver.environment.epsilon.value == 'water'

#doc = yaml.load(open('test-psolver.yaml','r'))
psolver.load(open('test-psolver.yaml','r'))
#psolver.load(doc)

print psolver.minimal()


False
True
environment:
  itermax: 0
setup:
  taskgroup_size: 1



In [25]:
cc = psolver.search('min')
print cc
var = cc[0]
print var, var.default,var.tomin()
var.set(4.0)
print var.parent.parent
print psolver.minimal()

Keys which match the most with "min"
[('minres', 'Convergence threshold of the loop'), ('pb_minres', 'Convergence criterion of the PBe loop')]
psolver.environment.minres: 1e-08 1e-08 default
psolver
environment:
  itermax: 0
  minres: 4.0
setup:
  taskgroup_size: 1



In [26]:
psolver.setup.accel.set('CUDA')
psolver.setup.use_gpu_direct.set(False)
print psolver.minimal()
print 30*'='
psolver.setup.accel.set('none')
print psolver.minimal()
print 30*'='
psolver.setup.use_gpu_direct.set(True)
print psolver.setup.use_gpu_direct.CONDITION
print psolver.minimal()

environment:
  itermax: 0
  minres: 4.0
setup:
  accel: CUDA
  taskgroup_size: 1
  use_gpu_direct: false

environment:
  itermax: 0
  minres: 4.0
setup:
  taskgroup_size: 1

{'MASTER_KEY': 'accel', 'WHEN': 'CUDA'}
environment:
  itermax: 0
  minres: 4.0
setup:
  taskgroup_size: 1



In [ ]:
 def runmenu(h, n, screen, menu, parent):
    "This function displays the appropriate menu and returns the option selected"
    optioncount = len(menu['options'])  # how many options in this menu
    # work out what text to display as the last menu option
    if parent is None:
        lastoption = "Exit (press 'Q' key)"
    else:
        lastoption = "Return to %s menu (press 'Q' key)" % parent['title']
    lastoption = "%d - %s" % (optioncount + 1, lastoption)
    #Size of the window
    screenH,screenW = screen.getmaxyx()

    #Build the title
    screen.border(0)
    # Title for this menu
    if menu['type'] == 'input':
        screen.addstr(2, 2, 'Toto', curses.A_BOLD)
    else:
        screen.addstr(2, 2, menu['title'], curses.A_BOLD)
    iy = 4
    if menu.has_key('DESCRIPTION'):
        for chain in menu['DESCRIPTION'].split('\n'):
            screen.addstr(iy, 2, chain, curses.A_BOLD)
            iy += 1
        iy += 1
    # Subtitle for this menu
    screen.addstr(iy, 2, menu['subtitle'], curses.A_BOLD)
    #Build a pad
    Wpad = screenW-2
    Hpad_screen = screenH-6
    menu_pad = curses.newpad(optioncount+1,max(Wpad,len(lastoption)+4))
    # Display all the menu items, showing the 'line' item highlighted
    for index in range(optioncount):
        menu_pad.addstr(index, 0, "%d - %s" % (index + 1, menu['options'][index]['title']), n)
    # Now display Exit/Return at bottom of menu
    menu_pad.addstr(optioncount, 0, lastoption, n)
    #Important to display menu_pad before the first key-press!!
    screen.refresh()

    # line is the zero-based index of the hightlighted menu option. Every time
    # runmenu is called, line returns to 0, when runmenu ends line
    # is returned and tells the program what opt$
    line = 0
    # control for while loop, let's you scroll through options until return
    # key is pressed then returns line to program
    x = None
    pos = 0
    # Loop until return key is pressed, or esc key
    while x != ord('\n') and x != ord('q') and x != ord('Q'):
        menu_pad.chgat(line,0,Wpad,h)
        menu_pad.refresh(pos,0,4,2,screenH-2,Wpad)
        x = screen.getch()  # Gets user input
        menu_pad.chgat(line,0,Wpad,n)
        # What is user input?
        # this line can be used only if optioncount is lower than 8
        if x >= ord('1') and x <= ord(str(min(optioncount + 1, 9))):
            # convert keypress back to a number, then subtract 1 to get index
            line = x - ord('0') - 1
        elif x == curses.KEY_DOWN:  # down arrow
            line += 1
            if line > optioncount:
                line = 0
                pos = 0
            if line-pos > Hpad_screen: pos += 1
        elif x == curses.KEY_UP:  # up arrow
            line += -1
            if line < 0:
                line = optioncount
                pos = line - Hpad_screen
            if line-pos < 0: pos -= 1
        elif x == ord('q') or x == ord('Q'):  # esc key
            line = optioncount
    # return index of the selected item
    return line


def dump_info(h, n, screen, menu, parent):
    """Dump some information in the screen and come back to previous menu when done.
    menu must contains 'title', 'subtitle' and 'DESCRIPTION'. """
    pos = 0
    oldpos = None
    x = None
    lastoption = "Press Enter or 'Q' key to come back to %s menu" % parent['title']
    screenH,screenW = screen.getmaxyx()
    #Build the height of the pad
    padH = 2*len(menu) + len(menu['DESCRIPTION'].split('\n'))
    for v in menu.values():
        padH += len(yaml.dump(v,default_flow_style=False).split('\n')) + 2
    dump_pad = curses.newpad(padH,screenW-2)
    m = menu.items()
    m.sort(menu_cmp, key= lambda s: s[0])
    # Title for this menu
    dump_pad.addstr(1, 2, m[0][1], curses.A_STANDOUT)
    # Subtitle for this menu
    dump_pad.addstr(3, 2, m[1][1], curses.A_BOLD)
    # Display all the menu items, showing the 'pos' item highlighted
    iy = 3
    for chain in m[2][1].split('\n'):
        iy += 1
        dump_pad.addstr(iy, 4, "%s" % chain, n)
    #Now all the other keys
    for (k,v) in m[3:]:
        if k == 'type' or k == 'COMMENT':
            pass
        else:
            iy += 1
            dump_pad.addstr(iy, 2, k, curses.A_BOLD)
            if isinstance(v,dict):
                text = yaml.dump(v,default_flow_style=False)
            else:
                text = str(v)
            for chain in text.split('\n'):
                iy += 1
                dump_pad.addstr(iy, 4, "%s" % chain, n)
            iy += 1
    # Now display Exit/Return at bottom of menu
    iy += 1
    dump_pad.addstr(iy, 2, " %s" % lastoption, h)
    #Display the information
    screen.border(0)
    screen.refresh()
    # Loop until return key is pressed
    pos = 0
    while x != ord('\n') and x != ord('q') and x != ord('Q'):
        dump_pad.refresh(pos,0,1,1,screenH-2,screenW-2)
        x = screen.getch()  # Gets user input
        if x == curses.KEY_DOWN:
            pos += 1
            pos = min(pos, iy - (screenH-4))
        elif x == curses.KEY_UP:
            pos -= 1
            pos = max(pos,0)
    # return index of the selected item
    return 0


def input_info(h, n, screen, menu, parent):
    """Search a key and dump info about it."""
    pos = 0
    oldpos = None
    x = None
    screenH,screenW = screen.getmaxyx()
    # Title for this menu
    screen.addstr(1, 2, 'Search a key', curses.A_STANDOUT)
    # Subtitle for this menu
    screen.addstr(3, 2, 'Please type a key:', curses.A_BOLD)
    #Display the information
    screen.border(0)
    screen.refresh()
    # Loop until return key is pressed
    key = ''
    posmin = 1 + len('Please type a key:') + 1
    blank = ' '*(screenW-posmin)
    pos = posmin
    while x != ord('\n'):
        pos = max(posmin,pos)
        if x == None:
            pass
        elif x >= ord('a') and x <= ord('z'):
            key += chr(x)
            pos += 1
        elif x == curses.KEY_BACKSPACE:
            key = key[:-1]
            pos -= 1
        screen.addstr(3,posmin,key,curses.A_NORMAL)
        screen.refresh()
        x = screen.getch()  # Gets user input
        #Remove the key and generate it (no pb with cursor)
        screen.addstr(3,posmin,blank,curses.A_NORMAL)
    #We have a key
    searchfor = key
    allkeys = []
    for f in files:
        allkeys += params[f].keys()
    matching_keys = closest_keys(allkeys, searchfor)
    #print 'matching', searchfor, 'in:', matching_keys
    #print "keys wich match the most with '%s'" % searchfor
    foundkeys = []
    text = '\n'
    for (i, ratio) in matching_keys:
        if ratio > 0.6 and i not in foundkeys:
            for f in find_files(params, i):
                text += '%s (Field: %s, similarity=%4.2f)\n' % (i,f,ratio) 
                text += yaml.dump(params[f][i], default_flow_style=False, explicit_start=False)
                foundkeys.append(i)
                text += '\n'
    #curses.endwin()
    #for chain in text.split('\n'):
    #    print text
    #sys.exit(0)
    #Build the pad
    padH = 3 + max(len(text.split('\n')),200)
    dump_pad = curses.newpad(padH,screenW-2)
    # Title for this menu
    dump_pad.addstr(1, 2, "Keys which match the most with '%s'" % searchfor, curses.A_STANDOUT)
    # Display all the menu items, showing the 'pos' item highlighted
    iy = 1
    if text == '':
        iy += 1
        dump_pad.addstr(iy, 4, "No key found", curses.A_NORMAL)
    else:   
        for chain in text.split('\n'):
            iy += 1
            if 'Field:' in chain:
                dump_pad.addstr(iy, 4, "%s" % chain, curses.A_BOLD)
            else:
                dump_pad.addstr(iy, 4, "%s" % chain, curses.A_NORMAL)
    lastoption = "Press Enter or 'Q' key to come back to '%s menu'" % parent['title']
    dump_pad.addstr(iy+1, 2, " %s" % lastoption, curses.A_STANDOUT)
    #Display the information
    screen.border(0)
    screen.refresh()
    # Loop until return key is pressed
    pos = 0
    x = None
    while x != ord('\n') and x != ord('q') and x != ord('Q'):
        dump_pad.refresh(pos,0,1,1,screenH-2,screenW-2)
        x = screen.getch()  # Gets user input
        if x == curses.KEY_DOWN:
            pos += 1
            pos = min(pos, iy - (screenH-4))
        elif x == curses.KEY_UP:
            pos -= 1
            pos = max(pos,0)
    return 0


def processmenu(h, n, screen, menu, parent=None):
    "This function calls showmenu and then acts on the selected item"
    optioncount = len(menu['options'])
    exitmenu = False
    while not exitmenu:  # Loop until the user exits the menu
        getin = runmenu(h, n, screen, menu, parent)
        try:
            whattodo = menu['options'][getin]['type']
        except:
            whattodo = None
        # the last option is always needed to exit the menu
        if getin == optioncount:
            exitmenu = True
        elif whattodo == COMMAND:
            curses.def_prog_mode()    # save current curses environment
            os.system('reset')
            screen.clear()  # clears previous screen
            os.system(menu['options'][getin]['command'])  # run the command
            # clears previous screen on key press and updates display based on pos
            screen.clear()
            curses.reset_prog_mode()   # reset to 'current' curses environment
            curses.curs_set(1)         # reset doesn't do this right
            curses.curs_set(0)
        elif whattodo == PROMPT:
            curses.def_prog_mode()    # save current curses environment
            # this is not the good place, as it is specific to the usage
            screen.clear()  # clears previous screen
            dump_info(h, n, screen, menu['options'][getin], menu)
            # clears previous screen on key press and updates display based on pos
            screen.clear()
            curses.reset_prog_mode()   # reset to 'current' curses environment
            curses.curs_set(1)         # reset doesn't do this right
            curses.curs_set(0)
        elif whattodo == MENU:
            # clears previous screen on key press and updates display based on pos
            screen.clear()
            # display the submenu
            processmenu(h, n, screen, menu['options'][getin], menu)
            # clears previous screen on key press and updates display based on pos
            screen.clear()
        elif whattodo == INPUT:
            # All pressed key is used
            curses.def_prog_mode()    # save current curses environment
            # this is not the good place, as it is specific to the usage
            screen.clear()  # clears previous screen
            input_info(h, n, screen, menu['options'][getin], menu)
            # clears previous screen on key press and updates display based on pos
            screen.clear()
            curses.reset_prog_mode()   # reset to 'current' curses environment
            curses.curs_set(1)         # reset doesn't do this right
            curses.curs_set(0)
        elif whattodo == EXITMENU:
            exitmenu = True

dico_menu = { 'title': 0, 'subtitle': 1, 'DESCRIPTION': 2, 'default': 3}
def menu_cmp(key_x,key_y):
    "Function compare to sort the keys in order to compose the menu"
    global dico_menu
    kx = dico_menu.get(key_x,key_x)
    ky = dico_menu.get(key_y,key_y)
    return cmp(kx,ky)
       # ertet
        # Initializes a new window for capturing key presses
        screen = curses.initscr()
        # Disables automatic echoing of key presses (prevents program from
        # input each key twice)
        curses.noecho()
        # Disables line buffering (runs each key as it is pressed rather than
        # waiting for the return key to pressed)
        curses.cbreak()
        # Lets you use colors when highlighting selected menu option
        curses.start_color()
        screen.keypad(1)  # Capture input from keypad
        screen.notimeout(1)  # Disables timeout for esc key
        # Change this to use different colors when highlighting
        # Sets up color pair #1, it does black text with white background
        curses.init_pair(1, curses.COLOR_BLACK, curses.COLOR_WHITE)
        # h is the coloring for a highlighted menu option
        h = curses.color_pair(1)
        # n is the coloring for a non highlighted menu option
        n = curses.A_NORMAL
        #Check if the height of window is enough
        H,W = screen.getmaxyx()
        if H > 6:
            processmenu(h, n, screen, menu)
        # VITAL! This closes out the menu system and returns you to the bash
        # prompt.
        curses.endwin()
        if H <= 6:
            sys.stdout.write("ERROR: The window is too thin!\n")
        os.system('clear')